In [ ]:
# TensorFlow 설치 (기본적으로 코랩에는 TensorFlow가 설치되어 있지만, 버전 업데이트가 필요할 수 있음)
%pip install tensorflow
%pip install tensorflow keras

# 모델을 사용하기 위한 추가 라이브러리 설치
%pip install h5py

In [6]:
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# 사전 학습된 Inception-ResNet-v2 모델 로드 (include_top=False)
base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(299, 299, 3))

# 기존 모델의 최상위 레이어를 제거하고 새로운 분류 레이어 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling 레이어 추가
x = Dense(1024, activation='relu')(x)  # 임의의 Dense 레이어 추가
predictions = Dense(2, activation='softmax')(x)  # 정상/비정상을 위한 2개의 클래스

# 새로운 모델 생성
model = Model(inputs=base_model.input, outputs=predictions)

# 전이 학습을 위해 기존의 레이어들을 학습되지 않도록 설정
# 전 학습된 모델의 초기 레이어들은 다양한 저수준의 특징(에지, 패턴 등)을 학습하기 때문에 다양한 이미지 분류 작업에 재사용될 수 있습니다.
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 제너레이터 설정
train_datagen = ImageDataGenerator(
    rescale=1.0/255,          # 모든 이미지의 픽셀 값을 0-1 범위로 스케일링
    shear_range=0.2,          # 이미지에 랜덤 전단 변환 적용
    zoom_range=0.2,           # 이미지에 랜덤 줌 변환 적용
    horizontal_flip=True      # 이미지를 랜덤으로 수평 반전
)

# 'datasets/Training' 디렉토리에서 이미지 불러오기
train_generator = train_datagen.flow_from_directory(
    '../../datasets/Training',      # 학습 데이터가 있는 디렉토리 경로
    target_size=(299, 299),   # 모델의 입력 크기에 맞게 이미지 크기 조정
    batch_size=32,            # 배치 크기
    class_mode='categorical'  # 다중 클래스 분류를 위해 categorical 모드 사용
)


Found 1615 images belonging to 2 classes.


In [ ]:
import os
import sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions


# Inception-ResNet-V2 모델 로드 (사전 학습된 가중치 사용)
model = InceptionResNetV2(weights='imagenet')

# 이미지가 저장된 로컬 폴더 경로
image_folder_path = os.path.join(os.getcwd(), '../../datasets/Validation/normal')

# 이미지 파일 목록 가져오기
if not os.path.exists(image_folder_path):
    print(f"Error: The path '{image_folder_path}' does not exist.")
else:
    image_files = [f for f in os.listdir(image_folder_path) if os.path.isfile(os.path.join(image_folder_path, f))]

    # 이미지 파일 목록에서 처음 20개만 선택
    first_twenty_images = image_files[:20]

    # 각 이미지에 대해 예측 수행
    for image_file in first_twenty_images:
        try:
            img_path = os.path.join(image_folder_path, image_file)
            img = Image.open(img_path)  # 이미지 열기
            plt.imshow(img)  # 이미지 표시
            plt.axis('off')  # 축 제거
            plt.show()  # 이미지 출력

            # 모델이 기대하는 입력 크기로 이미지 로드
            img = image.load_img(img_path, target_size=(299, 299))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)

            # 모델을 사용하여 예측
            preds = model.predict(x)

            # 예측 결과 디코딩 및 출력
            print(f'Image: {image_file}')
            print('Predicted:', decode_predictions(preds, top=3)[0])
            print('---------------------------------------')

        except Exception as e:
            print(f"Error processing {image_file}: {e}")